<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>PySpark e Apache Kafka Para Processamento de Dados em Batch e Streaming</font>
## <font color='blue'>Projeto 8</font>
### <font color='blue'>Simulação de Erros e Recuperação de Falhas em Multi-Node Kafka Cluster</font>
### <font color='blue'>Kafka Producer</font>

In [ ]:
# Imports
import os
import time
import json
import random
from confluent_kafka import Producer
from datetime import datetime, timedelta

In [ ]:
# Kafka Brokers
kafka_config = {'bootstrap.servers': 'kafka1:29092,kafka2:29093,kafka3:29094'}

In [ ]:
# Cria o Producer Kafka
producer = Producer(kafka_config)

In [ ]:
# Função de callback para relatar a entrega
def dsa_delivery_report(err, msg):
    if err is not None:
        print(f"Erro ao entregar mensagem: {err}")
    else:
        print(f"\nMensagem Entregue com Sucesso! "
              f"Tópico: {msg.topic()} | "
              f"Partição: {msg.partition()} | "
              f"Offset: {msg.offset()}")

In [ ]:
# Função para gerar os eventos
def dsa_gera_evento():
    return {
        "timestamp": datetime.now().isoformat(),
        "id_produto": random.randint(1, 100),
        "quantidade": random.randint(1, 10),
        "preco": round(random.uniform(10.0, 1000.0), 2)
    }

In [ ]:
# Nome do tópico Kafka
kafka_topic = 'dsa_p8_topico'

In [ ]:
# Loop para geração dos dados
print('\nIniciando a Produção dos Eventos\n')
for _ in range(100):
    
    try:

        # Gera o evento
        event = dsa_gera_evento()

        # Executa o producer
        producer.produce(kafka_topic,
                         key = str(event['timestamp']),
                         value = json.dumps(event),
                         callback = dsa_delivery_report)
        
        print(f"\nEvento Gerado: {event}")

        # O método producer.poll(0) é utilizado para processar eventos pendentes da fila do produtor
        producer.poll(0)  

        # Aguarda para gerar o próximo evento
        time.sleep(random.uniform(5, 10))
    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        time.sleep(5)

In [ ]:
producer.flush()

# Fim